In [24]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [25]:
from langchain.agents import tool
from typing import List, Dict, Annotated

In [26]:
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL

In [27]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [28]:
from langchain_teddynote.messages import AgentStreamParser
from langchain_teddynote.messages import AgentCallbacks
from bs4 import BeautifulSoup
import requests
import re

### AentStreamParser, AgentCallbacks

- tool_callback: 도구 호출 출력을 처리하는 함수
- observation_callback: 관찰(Observation) 출력을 처리하는 함수
- result_callback: 최종 답변 출력을 처리하는 함수

### 도구 정의

In [29]:
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""

    print(f'검색어: {query}')

    news_tool = GoogleNews()

    return news_tool.search_by_keyword(query, k=2)

In [30]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

In [31]:
@tool
def naver_news_crawl(news_url: str) -> str:                 # 네이버 뉴스기사를 크롤링하여 본문 내용을 리턴하는 도구
    """Crawls a 네이버(naver.com) news article and returns the body content."""

    response = requests.get(news_url)                       # HTTP GET 요청

    if response.status_code == 200:                         # 응답 200. 성공
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find('h2', id='title_area').get_text()
        content = soup.find('div', id='contents').get_text()    
        
        cleaned_title = re.sub(r'\n{2, }', '\n', title)
        cleaned_content = re.sub(r'\n{2, }', '\n', content)
    else:
        print(f'HTTP 요청 실패. {response.status_code}')
    
    return f'{cleaned_title}\n{cleaned_content}'

### Agent 프롬프트 작성

In [32]:
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (                                           
            "system",
            "You are a helpful assistant. "             
            "Make sure to use the `search_news` tool for searching keyword related news."
            "Be sure to use the 'python_repl_tool' tool when make python code."
            "Be sure to use the 'naver_news_crawl' tool when naver news crawlling."
        ),
        ("placeholder", "{chat_history}"),          # 이전 대화 내용을 넣을 곳을 잡아둔다.
        ("human", "{input}"),                       # 사용자 입력

        # 에이전트가 검색하는 과정들이나 내용 등을 끄적이는 메모장 같은 공간을 플레이스 홀더로 만들어준다
        ("placeholder", "{agent_scratchpad}"),       
    ]
)

In [33]:
tools = [search_keyword, python_repl_tool, naver_news_crawl]

### Agent 생성

In [34]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

agent = create_tool_calling_agent(llm, tools, prompt)

### AgentExecutor

In [35]:
agent_executor = AgentExecutor(
    agent=agent,                    # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                    # agent 가 사용할 수 있는 도구 목록
    verbose=True,
    max_iterations=10,              # 최대 10번 까지 반복
    max_execution_time=10,          # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

### 중간 단계를 출력하는 사용자 정의 함수 정의

In [36]:
def tool_callback(tool) -> None:
    print(f'===== 도구 호출 =====')
    print(f"사용된 도구 이름 : {tool.get('tool')}")
    print(f'===== 도구 호출 =====')

In [37]:
def obervation_callback(observation) -> None:
    print(f'===== 관찰 내용 =====')
    print(f"Observation : {observation.get('observation')[0]}")
    print(f'===== 관찰 내용 =====')

In [38]:
def result_callback(result: str) -> None:
    print(f'===== 최종 답변 =====')
    print(result)
    print(f'===== 최종 답변 =====')

In [39]:
agent_callbacks = AgentCallbacks(
    tool_callback=tool_callback, 
    observation_callback=obervation_callback, 
    result_callback=result_callback
)

### AgentStreamParser

In [40]:
agent_stream_parser = AgentStreamParser(agent_callbacks)

In [45]:
result = agent_executor.stream({'input': '랭체인에 대한 뉴스를 검색해주세요'})

for step in result:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : search_keyword
===== 도구 호출 =====

Invoking: `search_keyword` with `{'query': '랭체인'}`


검색어: 랭체인
[{'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTFBtcm1OMmpJOUxvdnlkd0M0dVFfaGhma3BrTDRaVEY2OXFwd2tfUTZHVkFNSkZJYkZLc0x2dzFWWDNldF9TaVhIU0NxSUxyMlpIRC1KNksyLWdNQQ?oc=5', 'content': '개발자를 위한 랭체인 기초 개념과 환경 설정법 - 요즘IT'}, {'url': 'https://news.google.com/rss/articles/CBMibkFVX3lxTE1qMC1EYjRZOEpMNVBEVVpSNm1BX3A4a3FrdmEyOTdDNWdjMGR1ZEgzWlpHZnRFZ1pSUXdKa3JnOXBQZU5qU3dKS3Q5VUNwLVZxRFZmbG1SZzJVSUoyM2lJbnZRbzhYYUJJcnFHWmp3?oc=5', 'content': '랭체인 LangChain 이란 무엇인가? | 인사이트리포트 | 삼성SDS - Samsung SDS'}]===== 관찰 내용 =====
Observation : {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTFBtcm1OMmpJOUxvdnlkd0M0dVFfaGhma3BrTDRaVEY2OXFwd2tfUTZHVkFNSkZJYkZLc0x2dzFWWDNldF9TaVhIU0NxSUxyMlpIRC1KNksyLWdNQQ?oc=5', 'content': '개발자를 위한 랭체인 기초 개념과 환경 설정법 - 요즘IT'}
===== 관찰 내용 =====
다음은 "랭체인"에 대한 최근 뉴스 기사입니다:

1. **[개발자를 위한 랭체인 기초 개념과 환경 설정법 - 

In [42]:
result = agent_executor.stream({'input': 'AI 관련 뉴스를 검색해 주세요.'})

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : search_keyword
===== 도구 호출 =====

Invoking: `search_keyword` with `{'query': 'AI'}`


검색어: AI
[{'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTFB1NnN6YVg0blIyNEhEUUp6SHNnaW1zdm1QZ3BIY2pnQzd2VmY4aTNiTVp0dlBPVTBRN3VsRm1KZHF6UUhUT1JrVUFTX0JISVVjaW4wdUVXd2FUZ9IBXEFVX3lxTFA0WFJJdHRJbU81QU9RWjJiTGxMTXVBUjBCSE16eFBZb3JPTG9PNWY0eVBvMklsVnRPZWdjU3NNZmFSZEppZmpmRjhYbzRMbWFwWG9JVW1WaVlPTFBJ?oc=5', 'content': '카카오 “오픈AI와 AI 에이전트 서비스 공동 개발, 연내 출시 목표” - 경향신문'}, {'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTFBtOEhLNk9sUDhrNVlvejZkOFc1RU9BUmZrWjhTSDM0TnJEM2xweFJ2LXdZSDUtNWJIWDBrVEllOTZCYU9Ganl1OC1Fdl84WDloYThSd0h3?oc=5', 'content': '작년 실적 네이버 \'방긋\'·카카오 \'찡긋\'…"AI 새 먹거리" 한 목소리 - ZD넷 코리아'}]===== 관찰 내용 =====
Observation : {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTFB1NnN6YVg0blIyNEhEUUp6SHNnaW1zdm1QZ3BIY2pnQzd2VmY4aTNiTVp0dlBPVTBRN3VsRm1KZHF6UUhUT1JrVUFTX0JISVVjaW4wdUVXd2FUZ9IBXEFVX3lxTFA0WFJJdHRJbU81QU9

In [43]:
result = agent_executor.stream({'input': '10 + 20을 출력하는 파이썬 코드를 작성하고 실행해주세요'})

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : python_repl_tool
===== 도구 호출 =====

Invoking: `python_repl_tool` with `{'code': 'print(10 + 20)'}`


30
===== 관찰 내용 =====
Observation : 3
===== 관찰 내용 =====
10 + 20의 결과는 30입니다.

> Finished chain.
===== 최종 답변 =====
10 + 20의 결과는 30입니다.
===== 최종 답변 =====


In [44]:
result = agent_executor.stream({'input': '네이버 뉴스를 크롤링 해주세요: https://n.news.naver.com/article/029/0002931985?cds=news_media_pc'})

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : naver_news_crawl
===== 도구 호출 =====

Invoking: `naver_news_crawl` with `{'news_url': 'https://n.news.naver.com/article/029/0002931985?cds=news_media_pc'}`


[설 밥상 화두] 비명 잠룡 중도층 민심잡기 본격화…이재명 일극체제 깰까



			설 연휴를 앞두고 이재명 더불어민주당 대표의 사법리스크가 현실화되고, 당의 지지율도 정체현상을 보이자 비명(비이재명)계 대권 잠룡들이 앞다퉈 쓴소리를 내며 존재감을 부각하고 있다. 이들은 공통적으로 이 대표와 친명(친이재명) 지도부가 중도·무당층의 마음을 얻지 못하고 있다고 판단하고, 그 틈새를 공략할 모양새다. 비명계 잠룡들이 대중 민심을 기반으로 이 대표 체제가 굳어진 당내에서 세를 결집할 수 있을 지 주목된다. 이 대표의 선거법 위반 사건 2심 재판부는 지난 23일 열린 첫 재판에서 "다음 달 26일 결심 공판을 하겠다"고 했다. 통상 결심 공판 이후 한달 뒤 선고 기일이 잡히는 만큼 3월 말 2심 선고가 나올 수 있다. 재판을 지연시켜 2심 선고 전에 대선을 치르는 최상의 시나리오가 사실상 물건너 간 셈이다. 이 대표는 1심에서 징역 1년에 집행유예 2년을 선고받았다. 이 판결이 2심에서도 그대로 유지되면, 이 대표의 대권가도에 상당한 걸림돌이 될 수 있다. 3심이 나오기 전 대선에 출마하더라도 이 문제가 계속 뜨거운 쟁점이 될 수 밖에 없기 때문이다. 이 대표와 민주당이 한덕수 대통령 권한대행의 탄핵을 밀어붙이고 윤석열 대통령 탄핵사유에서 내란죄를 삭제한 것도 헌재의 결정을 앞당겨 이같은 사법리스크를 최소화하기 위한 방책이었다.  최근 민심도 이 대표에게 녹록치 않다. 탄핵 바람을 타고 질주했던 민주당의 지지율은 하락하고, 여당인 국민의힘 지지율은 상승하면서 당 지지율에서 밀리는 상황이 